In [274]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [275]:
# df=pd.read_excel('Copper_Set.xlsx')
# print(df.info())

In [276]:
df1=df.copy()
df2=df1.copy()
df1.head()

,id,item_date,quantity tons,customer,country,status,item type,application,thickness,width,material_ref,product_ref,delivery date,selling_price
0,EC06F063-9DF0-440C-8764-0B0C05A4F6AE,20210401.0,54.151139,30156308.0,28.0,Won,W,10.0,2.00,1500.0,DEQ1 S460MC,1670798778,20210701.0,854.00
1,4E5F4B3D-DDDF-499D-AFDE-A3227EC49425,20210401.0,768.024839,30202938.0,25.0,Won,W,41.0,0.80,1210.0,0000000000000000000000000000000000104991,1668701718,20210401.0,1047.00
2,E140FF1B-2407-4C02-A0DD-780A093B1158,20210401.0,386.127949,30153963.0,30.0,Won,WI,28.0,0.38,952.0,S0380700,628377,20210101.0,644.33
3,F8D507A0-9C62-4EFE-831E-33E1DA53BB50,20210401.0,202.411065,30349574.0,32.0,Won,S,59.0,2.30,1317.0,DX51D+ZM310MAO 2.3X1317,1668701718,20210101.0,768.00
4,4E1C4E78-152B-430A-8094-ADD889C9D0AD,20210401.0,785.526262,30211560.0,28.0,Won,W,10.0,4.00,2000.0,2_S275JR+AR-CL1,640665,20210301.0,577.00


In [277]:
col_name={'quantity tons':'quantity_tons','item type':'item_type','delivery date':'delivery_date'}
df1.rename(columns=col_name, inplace=True)


In [278]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181673 entries, 0 to 181672
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id             181671 non-null  object 
 1   item_date      181672 non-null  float64
 2   quantity_tons  181673 non-null  object 
 3   customer       181672 non-null  float64
 4   country        181645 non-null  float64
 5   status         181671 non-null  object 
 6   item_type      181673 non-null  object 
 7   application    181649 non-null  float64
 8   thickness      181672 non-null  float64
 9   width          181673 non-null  float64
 10  material_ref   103754 non-null  object 
 11  product_ref    181673 non-null  int64  
 12  delivery_date  181672 non-null  float64
 13  selling_price  181672 non-null  float64
dtypes: float64(8), int64(1), object(5)
memory usage: 19.4+ MB


In [279]:
df1.shape

(181673, 14)

In [280]:
for col in df1.columns:
    print(f"{col} - {len(df1[col].unique())}") 


id - 181672
item_date - 253
quantity_tons - 181673
customer - 1170
country - 18
status - 10
item_type - 7
application - 31
thickness - 595
width - 1386
material_ref - 16564
product_ref - 33
delivery_date - 29
selling_price - 9796


In [281]:
df1.isnull().sum()

id                   2
item_date            1
quantity_tons        0
customer             1
country             28
status               2
item_type            0
application         24
thickness            1
width                0
material_ref     77919
product_ref          0
delivery_date        1
selling_price        1
dtype: int64

In [282]:
df1.item_date = df1.item_date.bfill()
df1.delivery_date = df1.delivery_date.bfill()
df1.id=df1.id.fillna('NA')
df1=df1.drop('material_ref',axis=1)
df1=df1.dropna()


In [283]:
print(df1.isnull().sum())
print(df1.shape)

id               0
item_date        0
quantity_tons    0
customer         0
country          0
status           0
item_type        0
application      0
thickness        0
width            0
product_ref      0
delivery_date    0
selling_price    0
dtype: int64
(181640, 13)


In [284]:
type_dict = {'id': 'str', 'quantity_tons': 'float32','customer':'int64','country':'int8','status':'category','item_type':'category','application':'int8',
            'thickness':'float32','width':'int32','product_ref':'int64','selling_price':'int64'}


In [285]:
df1=df1[df1.quantity_tons!='e']


In [286]:

df1 = df1.astype(type_dict)

df1.item_date = pd.to_datetime(df1.item_date,format='%Y%m%d', errors='coerce').dt.date
df1.delivery_date = pd.to_datetime(df1.delivery_date,format='%Y%m%d', errors='coerce').dt.date

In [287]:
df1.head()

,id,item_date,quantity_tons,customer,country,status,item_type,application,thickness,width,product_ref,delivery_date,selling_price
0,EC06F063-9DF0-440C-8764-0B0C05A4F6AE,2021-04-01,54.151138,30156308,28,Won,W,10,2.00,1500,1670798778,2021-07-01,854
1,4E5F4B3D-DDDF-499D-AFDE-A3227EC49425,2021-04-01,768.024841,30202938,25,Won,W,41,0.80,1210,1668701718,2021-04-01,1047
2,E140FF1B-2407-4C02-A0DD-780A093B1158,2021-04-01,386.127960,30153963,30,Won,WI,28,0.38,952,628377,2021-01-01,644
3,F8D507A0-9C62-4EFE-831E-33E1DA53BB50,2021-04-01,202.411072,30349574,32,Won,S,59,2.30,1317,1668701718,2021-01-01,768
4,4E1C4E78-152B-430A-8094-ADD889C9D0AD,2021-04-01,785.526245,30211560,28,Won,W,10,4.00,2000,640665,2021-03-01,577


In [288]:
df1.describe().T


,count,mean,std,min,25%,50%,75%,max
quantity_tons,181639.0,5.875974e+03,2.349294e+06,-2000.00,1.096869e+01,3.036395e+01,6.716187e+01,1.000000e+09
customer,181639.0,3.023251e+07,1.005614e+05,12458.00,3.019688e+07,3.020524e+07,3.028042e+07,3.040818e+07
country,181639.0,4.489299e+01,2.440423e+01,25.00,2.600000e+01,3.000000e+01,7.800000e+01,1.130000e+02
application,181639.0,2.561603e+01,1.775417e+01,2.00,1.000000e+01,1.500000e+01,4.100000e+01,9.900000e+01
thickness,181639.0,2.564145e+00,6.572500e+00,0.18,7.000000e-01,1.500000e+00,3.000000e+00,2.500000e+03
width,181639.0,1.295264e+03,2.616440e+02,1.00,1.180000e+03,1.250000e+03,1.500000e+03,2.990000e+03
product_ref,181639.0,4.740290e+08,7.175378e+08,611728.00,6.119930e+05,6.406650e+05,1.332077e+09,1.722208e+09
selling_price,181639.0,1.918292e+03,3.318258e+05,-1160.00,6.690000e+02,8.120000e+02,9.530000e+02,1.000010e+08


In [289]:
num=df1.select_dtypes(include=['int','float']).columns
for col in num:
    print(f"{col} -  {(df1[col] <= 0).sum()}")

quantity_tons -  4
customer -  0
thickness -  0
width -  0
product_ref -  0
selling_price -  8


In [293]:
df1 = df1[(df1.quantity_tons > 0) & (df1.selling_price > 0) ]
df1.describe().T

,count,mean,std,min,25%,50%,75%,max
quantity_tons,181627.0,5.876371e+03,2.349372e+06,0.00001,1.096917e+01,3.036408e+01,6.716187e+01,1.000000e+09
customer,181627.0,3.023251e+07,1.005641e+05,12458.00000,3.019688e+07,3.020524e+07,3.028042e+07,3.040818e+07
country,181627.0,4.489389e+01,2.440451e+01,25.00000,2.600000e+01,3.000000e+01,7.800000e+01,1.130000e+02
application,181627.0,2.561554e+01,1.775422e+01,2.00000,1.000000e+01,1.500000e+01,4.100000e+01,9.900000e+01
thickness,181627.0,2.550458e+00,2.976475e+00,0.18000,7.000000e-01,1.500000e+00,3.000000e+00,4.000000e+02
width,181627.0,1.295265e+03,2.616511e+02,1.00000,1.179500e+03,1.250000e+03,1.500000e+03,2.990000e+03
product_ref,181627.0,4.740123e+08,7.175321e+08,611728.00000,6.119930e+05,6.406650e+05,1.332077e+09,1.722208e+09
selling_price,181627.0,1.918423e+03,3.318367e+05,10.00000,6.690000e+02,8.120000e+02,9.530000e+02,1.000010e+08
